In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
import numpy as np

np.random.seed(42)

# Set your sample size
total_sampled_points = 5_000_000

# Automatically decide whether to sample with or without replacement
use_replacement = total_sampled_points > total_points

# Sample
all_indices = np.random.choice(total_points, size=total_sampled_points, replace=use_replacement)
dataset = dataset[all_indices]


# Optionally save the indices for reproducibility
# np.save("sampled_indices.npy", all_indices)


In [7]:
# --- Constants ---
SB = 10_000     # Small Batch
MB = 100_000    # Medium Batch
#LB = 1_000_000  # Large Batch
LB = total_sampled_points

SQ = 20         # Small Query
MQ = 100        # Medium Query
LQ = 500        # Large Query

# --- Generate test combinations ---
test_combinations = [(LB, SQ)]

In [17]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Sampled Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk from the *sampled* dataset
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [9]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

---------

**----------KD Tree---------------------**

In [10]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**---------KD Tree Hybrid------------**

**----------------KD Run 1------------------------------**

In [20]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 61.4787 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 30.09it/s]

Total Search Time: 1.722602 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 61.4787 sec, Total Search Time: 1.722602 sec, Total Time: 63.2013 sec

Total Accumulated Time (All Batches): 63.2013 sec


**-------------KD Hybrid Run 2---------------------**

In [15]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 64.8789 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 28.58it/s]

Total Search Time: 1.731714 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 64.8789 sec, Total Search Time: 1.731714 sec, Total Time: 66.6106 sec

Total Accumulated Time (All Batches): 66.6106 sec


**--------------Kd Hybrid Run 3-----------------------**

In [18]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 64.6597 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 30.05it/s]

Total Search Time: 1.692975 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 64.6597 sec, Total Search Time: 1.692975 sec, Total Time: 66.3527 sec

Total Accumulated Time (All Batches): 66.3527 sec


**----------KD Hybrid Avg and Std------**

In [21]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Build Time: 61.4787 sec, Total Search Time: 1.722602 sec, Total Time: 63.2013 sec',
    'Build Time: 64.8789 sec, Total Search Time: 1.731714 sec, Total Time: 66.6106 sec',
    'Build Time: 64.6597 sec, Total Search Time: 1.692975 sec, Total Time: 66.3527 sec'
]

# Regular expressions to extract the times
build_times = []
search_times = []
total_times = []

for line in results:
    build_match = re.search(r'Build Time:\s*([\d.]+)', line)
    search_match = re.search(r'Total Search Time:\s*([\d.]+)', line)
    total_match = re.search(r'Total Time:\s*([\d.]+)', line)

    if build_match and search_match and total_match:
        build_times.append(float(build_match.group(1)))
        search_times.append(float(search_match.group(1)))
        total_times.append(float(total_match.group(1)))

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Build Time", build_times)
stats("Search Time", search_times)
stats("Total Time", total_times)


Build Time - Mean: 63.6724 sec, Std Dev: 1.9030 sec, CV: 2.99%
Search Time - Mean: 1.7158 sec, Std Dev: 0.0203 sec, CV: 1.18%
Total Time - Mean: 65.3882 sec, Std Dev: 1.8983 sec, CV: 2.90%


**-----------------Original KD Tree--------**

**--------Run 1------------------**

In [13]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 157.4492 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 27.40it/s]

Total Search Time: 0.732303 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 157.4492 sec, Total Search Time: 0.732303 sec, Total Time: 158.1816 sec

Total Accumulated Time (All Batches): 158.1816 sec


**---------Run 2---------------**

In [16]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 156.1774 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 28.97it/s]

Total Search Time: 0.692469 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 156.1774 sec, Total Search Time: 0.692469 sec, Total Time: 156.8699 sec

Total Accumulated Time (All Batches): 156.8699 sec


**-----------------Run 3-----------------**

In [22]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.000000

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the KD Tree from scratch
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt KD Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt KD Tree with 5,000,000 points.
Build Time: 154.8618 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 28.99it/s]

Total Search Time: 0.691769 sec

=== Summary of KD Tree Scaling Test ===
Points: 5,000,000, Build Time: 154.8618 sec, Total Search Time: 0.691769 sec, Total Time: 155.5536 sec

Total Accumulated Time (All Batches): 155.5536 sec


**-------Original KD Avg and STD-------**

In [23]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Build Time: 157.4492 sec, Total Search Time: 0.732303 sec, Total Time: 158.1816 sec',
    'Build Time: 156.1774 sec, Total Search Time: 0.692469 sec, Total Time: 156.8699 sec',
    'Build Time: 154.8618 sec, Total Search Time: 0.691769 sec, Total Time: 155.5536 sec'
]

# Regular expressions to extract the times
build_times = []
search_times = []
total_times = []

for line in results:
    build_match = re.search(r'Build Time:\s*([\d.]+)', line)
    search_match = re.search(r'Total Search Time:\s*([\d.]+)', line)
    total_match = re.search(r'Total Time:\s*([\d.]+)', line)

    if build_match and search_match and total_match:
        build_times.append(float(build_match.group(1)))
        search_times.append(float(search_match.group(1)))
        total_times.append(float(total_match.group(1)))

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Build Time", build_times)
stats("Search Time", search_times)
stats("Total Time", total_times)


Build Time - Mean: 156.1628 sec, Std Dev: 1.2938 sec, CV: 0.83%
Search Time - Mean: 0.7055 sec, Std Dev: 0.0232 sec, CV: 3.29%
Total Time - Mean: 156.8684 sec, Std Dev: 1.3140 sec, CV: 0.84%
